In [10]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

URL="https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile"
def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  

def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME,'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  
              
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  
            
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  
            
            bus_name_elements = driver.find_elements(By.XPATH,"//div[@class='travels lh-24 f-bold d-color']")
            bus_type_elements = driver.find_elements(By.XPATH,"//div[@class='bus-type f-12 m-top-16 l-color evBus']")
            departing_time_elements = driver.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
            duration_elements = driver.find_elements(By.XPATH,"//div[@class='dur l-color lh-24']")
            arrival_elements= driver.find_elements(By.XPATH,"//div[@class='bp-time f-19 d-color disp-Inline']")
            rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            fare_elements = driver.find_elements(By.CSS_SELECTOR,"span.f-19.f-bold")
            seat_availability_elements = driver.find_elements(By.CSS_SELECTOR,".seat-left")
            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Arrival": arrival_elements[i].text,
                    "Rating": rating_elements[i].text if i < len(rating_elements) else '0',
                    "Fare": fare_elements[i].text,
                    "Seat_Available": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

all_bus_details = []

def scrape_all_pages():
    for page in range(1, 6):  # There are 5 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs ')][text()='{page}']")))
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5) 
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
           
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            
         
        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

scrape_all_pages()

df = pd.DataFrame(all_bus_details)

df.to_csv('kadamba.csv', index=False)

driver.quit()

Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/solapur-to-goa: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7B0086CB5+28821]
	(No symbol) [0x00007FF7AFFF3840]
	(No symbol) [0x00007FF7AFE9578A]
	(No symbol) [0x00007FF7AFEE91BE]
	(No symbol) [0x00007FF7AFEE94AC]
	(No symbol) [0x00007FF7AFF32647]
	(No symbol) [0x00007FF7AFF0F33F]
	(No symbol) [0x00007FF7AFF2F412]
	(No symbol) [0x00007FF7AFF0F0A3]
	(No symbol) [0x00007FF7AFEDA778]
	(No symbol) [0x00007FF7AFEDB8E1]
	GetHandleVerifier [0x00007FF7B03BFCAD+3408013]
	GetHandleVerifier [0x00007FF7B03D741F+3504127]
	GetHandleVerifier [0x00007FF7B03CB5FD+3455453]
	GetHandleVerifier [0x00007FF7B014BDBB+835995]
	(No symbol) [0x00007FF7AFFFEB5F]
	(No symbol) [0x00007FF7AFFFA814]
	(No symbol) [0x00007FF7AFFFA9AD]
	(No symbol) [0x00007FF7AFFEA199]
	BaseThreadInitThunk [0x00007FFDEEED7374+20]
	RtlUserThreadStart [0x00007FFDEFDFCC91+33]

Error occurred while scraping bus details for https://www.redbus.in/bus-tickets

NameError: name 'driver' is not defined

In [12]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Arrival,Rating,Fare,Seat_Available
0,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,IntrCity SmartBus,Bharat Benz A/C Seater /Sleeper (2+1),21:15,10h 50m,08:05,4.5,1298,13 Seats available
1,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,IntrCity SmartBus,AC Sleeper (2+1),22:15,10h 30m,08:45,4.1,1103,12 Seats available
2,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,Laxmi-Princess Nova,Non A/C Seater / Sleeper (2+1),20:00,11h 15m,07:15,4.1,999,10 Seats available
3,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,Ashray Travels,Bharat Benz A/C Sleeper (2+1),21:05,11h 25m,08:30,4.2,1599,9 Seats available
4,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,IntrCity SmartBus,Bharat Benz A/C Sleeper (2+1),22:30,11h 05m,09:35,4.2,1440,11 Seats available
...,...,...,...,...,...,...,...,...,...,...
485,Goa to Shirdi,https://www.redbus.in/bus-tickets/goa-to-shirdi,Sea Bird Tourist,A/C Sleeper (2+1),19:00,49h 30m,20:30,3.0,11000,15 Seats available
486,Calangute (goa) to Goa,https://www.redbus.in/bus-tickets/calangute-go...,UNIQUE TOURS AND TRAVELS,A/C Seater Push Back (2+1),08:30,11h 30m,20:00,3.2,400,17 Seats available
487,Mopa Airport to Panaji,https://www.redbus.in/bus-tickets/mopa-airport...,Kadamba Transport Corporation Limited (KTCL) -...,A/C Seater (2+3),12:30,00h 55m,13:25,3.6,200,45 Seats available
488,Belagavi to Saquelim,https://www.redbus.in/bus-tickets/belagavi-to-...,Kadamba Transport Corporation Limited (KTCL) -...,Non AC Seater 2+2,16:00,02h 15m,18:15,3.6,140,25 Seats available


In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

URL="https://www.redbus.in/online-booking/chandigarh-transport-undertaking-ctu"
def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  

def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME,'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  
              
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  
            
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  
            
            bus_name_elements = driver.find_elements(By.XPATH,"//div[@class='travels lh-24 f-bold d-color']")
            bus_type_elements = driver.find_elements(By.XPATH,"//div[@class='bus-type f-12 m-top-16 l-color evBus']")
            departing_time_elements = driver.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
            duration_elements = driver.find_elements(By.XPATH,"//div[@class='dur l-color lh-24']")
            arrival_elements= driver.find_elements(By.XPATH,"//div[@class='bp-time f-19 d-color disp-Inline']")
            rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            fare_elements = driver.find_elements(By.CSS_SELECTOR,"span.f-19.f-bold")
            seat_availability_elements = driver.find_elements(By.CSS_SELECTOR,".seat-left")
            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Arrival": arrival_elements[i].text,
                    "Rating": rating_elements[i].text if i < len(rating_elements) else '0',
                    "Fare": fare_elements[i].text,
                    "Seat_Available": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

all_bus_details = []

def scrape_all_pages():
    for page in range(1, 6):  # There are 5 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs ')][text()='{page}']")))
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5) 
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
           
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            
         
        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

scrape_all_pages()

df = pd.DataFrame(all_bus_details)

df.to_csv('chandigarh.csv', index=False)

driver.quit()

Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/sujanpur-to-chandigarh: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7B0086CB5+28821]
	(No symbol) [0x00007FF7AFFF3840]
	(No symbol) [0x00007FF7AFE9578A]
	(No symbol) [0x00007FF7AFEE91BE]
	(No symbol) [0x00007FF7AFEE94AC]
	(No symbol) [0x00007FF7AFF32647]
	(No symbol) [0x00007FF7AFF0F33F]
	(No symbol) [0x00007FF7AFF2F412]
	(No symbol) [0x00007FF7AFF0F0A3]
	(No symbol) [0x00007FF7AFEDA778]
	(No symbol) [0x00007FF7AFEDB8E1]
	GetHandleVerifier [0x00007FF7B03BFCAD+3408013]
	GetHandleVerifier [0x00007FF7B03D741F+3504127]
	GetHandleVerifier [0x00007FF7B03CB5FD+3455453]
	GetHandleVerifier [0x00007FF7B014BDBB+835995]
	(No symbol) [0x00007FF7AFFFEB5F]
	(No symbol) [0x00007FF7AFFFA814]
	(No symbol) [0x00007FF7AFFFA9AD]
	(No symbol) [0x00007FF7AFFEA199]
	BaseThreadInitThunk [0x00007FFDEEED7374+20]
	RtlUserThreadStart [0x00007FFDEFDFCC91+33]

Error occurred while scraping bus details for https://www.redbus.in/bus

NameError: name 'driver' is not defined

In [4]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Arrival,Rating,Fare,Seat_Available
0,Delhi to Chandigarh,https://www.redbus.in/bus-tickets/delhi-to-cha...,HRTC - 551,A/C Executive (2+3),15:30,05h 55m,21:25,3.9,400,41 Seats available
1,Delhi to Chandigarh,https://www.redbus.in/bus-tickets/delhi-to-cha...,HRTC - 525,Ordinary 3+2 Non AC Seater,16:50,05h 50m,22:40,3.9,290,27 Seats available
2,Delhi to Chandigarh,https://www.redbus.in/bus-tickets/delhi-to-cha...,HRTC - 526,Ordinary 3+2 Non AC Seater,17:00,06h 00m,23:00,3.9,275,33 Seats available
3,Delhi to Chandigarh,https://www.redbus.in/bus-tickets/delhi-to-cha...,HRTC - 587,Ordinary 3+2 Non AC Seater,18:25,04h 35m,23:00,3.9,285,33 Seats available
4,Delhi to Chandigarh,https://www.redbus.in/bus-tickets/delhi-to-cha...,HRTC - 541,Ordinary 3+2 Non AC Seater,19:14,05h 16m,00:30,3.9,290,23 Seats available
...,...,...,...,...,...,...,...,...,...,...
442,Chandigarh to Haridwar,https://www.redbus.in/bus-tickets/chandigarh-t...,SHRI KRISHNA TRAVELS (JAI SHREE GANESH YATRA CO.),VE A/C Seater / Sleeper (2+1),23:30,06h 05m,05:35,3.7,1399,10 Seats available
443,Chandigarh to Haridwar,https://www.redbus.in/bus-tickets/chandigarh-t...,Laxmi holidays,VE A/C Semi Sleeper (2+2),23:55,06h 35m,06:30,4.7,949,19 Seats available
444,Chandigarh to Haridwar,https://www.redbus.in/bus-tickets/chandigarh-t...,SHRI KRISHNA TRAVELS (JAI SHREE GANESH YATRA CO.),VE A/C Seater / Sleeper (2+1),23:55,05h 40m,05:35,4.1,1299,10 Seats available
445,Chandigarh to Haridwar,https://www.redbus.in/bus-tickets/chandigarh-t...,City Land Travels,Volvo A/C Semi Sleeper (2+2),23:55,05h 06m,05:01,3.8,1250,24 Seats available
